1 subject.
No feature extraction.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

files = os.listdir('d\\')

list_of_dfs = []
for file in files:
    with open(os.path.join('d\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
        
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        allData = allData.merge(aux, left_index=True, right_index=True)
        
        list_of_dfs.append(allData)
big_df = pd.concat(list_of_dfs, ignore_index=True)#ignore_index to reset index of big_df

data = big_df.drop(['arousal', 'dominance', 'liking'], axis=1)
data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence']

data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature'],axis = 'columns', inplace=True)

data

,FP1,AF3,F7,F3,FC1,FC5,T7,C3,CP1,CP5,...,T8,FC6,FC2,F4,F8,AF4,Fp2,Fz,Cz,valence
0,0.948232,10.260175,1.013050,-7.658428,-1.811108,11.011411,3.026008,-2.380048,3.978952,-9.657708,...,1.134884,1.435263,1.388486,4.239575,4.888393,0.596471,0.589618,-2.276449,-0.109300,happy
1,1.653335,12.795443,-1.067832,-3.267558,-4.783876,7.402976,2.676232,-3.614201,-1.434440,-0.906298,...,1.824904,-1.665933,-1.577337,4.557239,6.007259,-1.881391,-4.831903,-1.739787,-6.518661,happy
2,3.013726,10.426192,3.908249,0.701542,-0.522649,1.120469,2.046996,-4.286566,-6.174767,9.408599,...,-4.090714,1.008985,-6.161669,0.636801,2.921478,-4.484906,-8.186728,0.555901,-11.727187,happy
3,1.495061,8.229207,6.094405,2.959722,1.299854,-0.832024,2.192056,-5.170547,-6.571542,14.101073,...,-2.502785,0.562838,-4.305524,-2.965047,-0.679860,-4.483488,-7.905437,-1.168129,-9.051847,happy
4,-1.264836,3.751782,4.145906,3.459897,-0.916779,-0.784404,-4.694002,-5.332026,-7.793136,16.284187,...,6.397443,-3.709545,-0.162134,3.248115,-1.103246,-2.547386,-4.419073,-1.319219,-4.072682,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322555,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.214036,-0.050429,...,0.068903,0.140944,-0.272806,-0.101962,0.036677,-0.007930,0.081379,-0.081538,-0.009341,happy
322556,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.215036,-0.050429,...,0.068903,0.140944,-0.272806,-0.101962,0.036677,-0.007930,0.081379,-0.081538,-0.009341,happy
322557,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.215036,-0.050429,...,0.068903,0.140944,-0.272806,-0.101962,0.036677,-0.007930,0.081379,-0.082538,-0.009341,happy
322558,-0.097608,-0.062015,-0.151092,0.013060,-0.046585,-0.060403,0.149920,0.000826,0.214036,-0.050429,...,0.068903,0.140944,-0.272806,-0.101962,0.036677,-0.007930,0.081379,-0.081538,-0.009341,happy


In [2]:
from sklearn.model_selection import train_test_split

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)

# Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
clf.fit(xTrain, yTrain)

<ipython-input-3-353746cc4ea1>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    4.7s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:   11.6s finished


RandomForestClassifier(max_depth=2, n_jobs=5, random_state=0, verbose=1)

In [4]:
predict = clf.predict(xTest)

from sklearn.metrics import accuracy_score

accuracy_score(yTest, predict)

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished


0.6491505456349206

# SVM

In [7]:
from sklearn import svm
from sklearn.svm import LinearSVC

svm_clf = svm.LinearSVC()
svm_clf.fit(xTrain, np.ravel(yTrain))
svm_predict = svm_clf.predict(xTest)

C:\Python39\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [8]:
accuracy_score(yTest, svm_predict)

0.5456039186507936

# kNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 7

knn = KNeighborsClassifier(n_neighbors)
knn.fit(xTrain, np.ravel(yTrain))

print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(xTrain, yTrain)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(xTest, yTest)))

Accuracy of K-NN classifier on training set: 0.88
Accuracy of K-NN classifier on test set: 0.81


In [10]:
knn_predict = knn.predict(xTest)
accuracy_score(yTest, knn_predict)

0.8100818452380952